In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import SPTCata as spt
import matplotlib.pyplot as plt
import numpy as np
import os, sys
import re
from collections import defaultdict

# for fitting
from scipy.optimize import curve_fit

# 2019-04-05 MSD and diffusion analysis
I proceed with the analysis of the MSD and diffusion coefficients.

## Preliminaries

In [ ]:
# this is the function that we will use to calculate the MSD
def msd_t (tracks) :
    
    # init the variables
    ntracks = len(tracks)
    lengths = [t.shape[0] for t in tracks]
    max_t = int(max(lengths)/2)

    # make a big matrix with all the mean values, fill it with NaNs, and then
    # fill it with the mean of the MSD for each track
    MSD = np.zeros((ntracks, max_t))
    MSD.fill(np.nan)

    # iterate through the all the tracks
    for i, track in enumerate(tracks) :
        
        # number of time steps in the track
        T = track.shape[0]
        
        # now initialize the Delta matrix, which contains the
        # squared differences between the particles' positions
        # at different time delays
        Nt = int(T/2)
        delta = np.zeros((Nt,Nt))
        
        # calculate the MSD for the single track
        for delay in range(1,Nt+1) :
            for t0 in range (Nt) :
                t1 = t0 + delay
                pos1 = track[t1,:]
                pos0 = track[t0,:]
                delta[delay-1,t0] = np.sum((pos1-pos0)**2)
        
        # append the mean of this delta matrix to the big delta matrix
        msd = delta.mean(axis = 1)
        MSD[i, 0:len(msd)] = msd
    
    # return the MSD
    return np.nanmean(MSD, axis = 0)

## Load data

In [ ]:
# get all the directory names where we have 10 Spots tracks
datadirs = []
for d, sd, fs in os.walk('../data') :
    result = re.match('.*10 [Ss]pot.*', d)
    if result :
        datadirs.append(d)

In [ ]:
# cycle through all the directories and do the analysis
experiments = defaultdict(list)
data = defaultdict(list)
for datadir in datadirs :
    treatment = datadir.split('/')[-3].replace("_1000frames", "")
    experiment = spt.SPT(datadir)
    experiments[treatment].append(experiment)
    trajectories = experiment.trajectory_links
    data[treatment].extend(trajectories)

## MSD

To do the MSD analysis, I use the `msd_t` function that I defined before.

In [ ]:
d = msd_t(data['R5020'])

I can then plot the MSD.

In [ ]:
T = d.shape[0]
plt.plot(d)
alpha = 0.6
t = np.arange(T)
D = 0.03
plt.plot(t, D*t**alpha)

## Diffusion coefficient

To calculate the diffusion coefficient, I'll use the fitting of the distribution of displacements.

First, I gather the information on the square displacements in different treatments.

In [ ]:
# calculate the distribution of r2 for all the tracks
displacements2 = defaultdict(list)
for treatment, tracks in experiments.items() :
    for track in tracks :
        displacements2[treatment].extend(np.array(track.displacement_spots)**2)

I define then two competitor models for the diffusion: a simple one-species diffusion model, and a two-species diffusion model.

In [ ]:
def one_species_diffusion_model(x, *p):
    D = p
    return 1-np.exp(-x/D)

def two_species_diffusion_model(x, *p) :
    D1, D2, f1 = p
    return 1-f1*np.exp(-x/D1)-(1-f1)*np.exp(-x/D2)

Now we're ready to perform the fits.

In [ ]:
N = 200
for treatment, displacement2 in displacements2.items() :
    h, bins = np.histogram(displacement2, bins=N)
    delta = bins[1]-bins[0]
    x = np.zeros(N+1)
    y = np.zeros(N+1)
    x = bins + delta/2.
    y[1:] = np.cumsum(h)/sum(h)
    
    # one species diffusion model fit
    p0 = 0.02
    one_species_coeff, one_species_var_matrix = curve_fit(one_species_diffusion_model, x, y, p0=p0)
    
    # two species diffusion model fit
    p0 = [0.02, 0.1, 0.5]
    two_species_coeff, two_species_var_matrix = curve_fit(two_species_diffusion_model, x, y, p0=p0)
    
    plt.plot(x, y, 'o', markersize=3, label='Data')
    plt.plot(x, one_species_diffusion_model(x, one_species_coeff[0]), label='One species fit')
    plt.plot(x, two_species_diffusion_model(x, two_species_coeff[0],
                                               two_species_coeff[1],
                                               two_species_coeff[2]), label='Two species fit')
    plt.title(treatment, fontsize=18)
    plt.legend(loc='lower right')
    plt.show()
    
    print("Treatment: %s"%(treatment))
    print("\tOne species fit: D = %.3f"%(one_species_coeff[0]))
    print("\tTwo species fit: D1 = %.3f, D2 = %.3f, f1 = %.3f"%(two_species_coeff[0],
                                                                two_species_coeff[1],
                                                                two_species_coeff[2]))
    